# For  LDM-503-EFDb  

Initialize EFDs Summit and USDF

In [5]:
from lsst_efd_client import EfdClient, resample

client = EfdClient('summit_efd')
client.output = 'dataframe'
cl=client.influx_client



usdf_client = EfdClient('usdf_efd')
usdf_client.output = 'dataframe'
usdf_cl=usdf_client.influx_client


In [ ]:
## Pick five topics

In [9]:
import random
topics = await client.get_topics()

selected_topics = []
results = {}
loc = "summit"

day = '2023-05-20'

# want to selct 5 topics randomly but with messages so randomize all indexes 
randoms = random.sample(range(0,len(topics)),len(topics))
for r in randoms:
   topic = topics[r]
   result = []
   if len(selected_topics) < 5:
      query=f'''SELECT * FROM "{topic}" WHERE time >  '{day}T00:00:00.000Z' and time < '{day}T00:00:30.000Z' '''
      result = await cl.query(query)

   if len(result) > 20:
        print (f"{topic} had {len(result)} messages in first 30 min of {day}")
        selected_topics.append(topic)
        results[topic] = result
        if len(selected_topics) > 4:
            break


print(f"Random selction of five '{loc}' topics {selected_topics} with messages on {day}")

lsst.sal.GIS.logevent_heartbeat had 30 messages in first 30 min of 2023-05-20
lsst.sal.ATOODS.logevent_heartbeat had 30 messages in first 30 min of 2023-05-20
lsst.sal.Test.logevent_heartbeat had 30 messages in first 30 min of 2023-05-20
lsst.sal.ESS.relativeHumidity had 87 messages in first 30 min of 2023-05-20
lsst.sal.ESS.logevent_heartbeat had 329 messages in first 30 min of 2023-05-20
Random selction of five 'summit' topics ['lsst.sal.GIS.logevent_heartbeat', 'lsst.sal.ATOODS.logevent_heartbeat', 'lsst.sal.Test.logevent_heartbeat', 'lsst.sal.ESS.relativeHumidity', 'lsst.sal.ESS.logevent_heartbeat'] with messages on 2023-05-20


In [ ]:
## Utility function to compare reusults

In [10]:
def cmp(topic, result, sresult):
   print (f"{topic} had {len(result)} messages - summit had {len(sresult)}")
   if len(result) != len(sresult):
        print (f"{topic} does not have the same number of results")
   else:
        for k in sresult.keys():
            if result[k].all() != sresult[k].all():
                print (f"{topic} {key} does not match")


In [ ]:
## Get dataframes

In [24]:
day = '2023-06-12'
day2 = '2023-06-18' # test gives large window 6 days

data = {}

for topic in selected_topics:
    query=f'''SELECT * FROM "{topic}" WHERE time >  '{day}T00:00:00.000Z' and time < '{day2}T23:59:59.000Z' '''
    result = await cl.query(query)
    usdf_result = await usdf_cl.query(query)
    cmp(topic, usdf_result, result)
    data[topic] = usdf_result
    

lsst.sal.GIS.logevent_heartbeat had 262720 messages - summit had 262660
lsst.sal.GIS.logevent_heartbeat does not match
lsst.sal.ATOODS.logevent_heartbeat had 259254 messages - summit had 259214
lsst.sal.ATOODS.logevent_heartbeat does not match
lsst.sal.Test.logevent_heartbeat had 261295 messages - summit had 261243
lsst.sal.Test.logevent_heartbeat does not match
lsst.sal.ESS.relativeHumidity had 735137 messages - summit had 734512
lsst.sal.ESS.relativeHumidity does not match
lsst.sal.ESS.logevent_heartbeat had 2888679 messages - summit had 2886217
lsst.sal.ESS.logevent_heartbeat does not match


In [ ]:
## Reliability 

In [ ]:
for topic in selected_topics:
    seqnum = data[topic]['private_seqnum']
    print (seqnum)
    
    

In [22]:
sday = '2023-06-1'     

for i in range(0,7):
    day = f"{sday}{i}"
    print(f"30 min on {day}")
    for topic in selected_topics:
        query=f'''SELECT * FROM "{topic}" WHERE time >  '{day}T00:00:00.000Z' and time < '{day}T00:00:30.000Z' '''
        result = await cl.query(query)
        usdf_result = await usdf_cl.query(query)
        cmp(topic, usdf_result, result)



30 min on 2023-06-10
lsst.sal.GIS.logevent_heartbeat had 0 messages - summit had 0
lsst.sal.ATOODS.logevent_heartbeat had 0 messages - summit had 0
lsst.sal.Test.logevent_heartbeat had 0 messages - summit had 0
lsst.sal.ESS.relativeHumidity had 0 messages - summit had 0
lsst.sal.ESS.logevent_heartbeat had 0 messages - summit had 0
30 min on 2023-06-11
lsst.sal.GIS.logevent_heartbeat had 0 messages - summit had 0
lsst.sal.ATOODS.logevent_heartbeat had 0 messages - summit had 0
lsst.sal.Test.logevent_heartbeat had 0 messages - summit had 0
lsst.sal.ESS.relativeHumidity had 0 messages - summit had 0
lsst.sal.ESS.logevent_heartbeat had 0 messages - summit had 0
30 min on 2023-06-12
lsst.sal.GIS.logevent_heartbeat had 0 messages - summit had 0
lsst.sal.ATOODS.logevent_heartbeat had 0 messages - summit had 0
lsst.sal.Test.logevent_heartbeat had 0 messages - summit had 0
lsst.sal.ESS.relativeHumidity had 0 messages - summit had 0
lsst.sal.ESS.logevent_heartbeat had 0 messages - summit had 0
3